In [13]:
import tensorflow as tf
import numpy as np

# preprocessed data
#from datasets.twitter import data
import data
import data_utils
#from handleQuery import *

In [14]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [15]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [16]:
import seq2seq_wrapper

In [17]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/Users/subash/Desktop/cornellMovieConversationBot/tensorFlow-seq2seq/seq2seq_wrapper.py'>

In [18]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [19]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)

train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [20]:
#sess = model.train(train_batch_gen, val_batch_gen)

In [21]:
sess = model.restore_last_session()

In [22]:
input_ = test_batch_gen.__next__()[0]
print (input_.shape)
print (input_[0])

output = model.predict(sess, input_)
print(output.shape)

(20, 256)
[   1  199  112   41  125    1  234  103   41 2423 4502    2   28    4  424
  122    1   67   28  314   77    4  801 7859   88    2   28   87    4 3324
  342  832    4 3680  136 1010 1264    7 4898  456    7  197  174    3   75
  135   27  122   49   56  239    4  686    4    4    4   23    5   26  378
  224 2121    5  726  122  215  248  226  118 1599   23  131  196 1324   36
    5  128   17  533  305  152    4  103    4 1073    2  186  561   63   56
   93   15 4214   50    4 1758  418   16   31   57   32    7  199    9    1
  949    4 1281   21    7    5   17    5   39  503   50  229   16   28   13
   42   35  381    5   14  229   32    4    1  148   27  130    1  160 1516
    5  192    4   60 2591 3121    7  222  236    5    1   88  166   17    1
 1358   27    7   54  241    5    4 1758   65    1 2327   57   84 1275  160
   15    1 3452    5   75   65   56   27 1457    4    8   17  189  254  581
   20   15    2   15   15   39  430    8   37  523    1  343  143    1  211
  

In [23]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)


q : [unk want to have sex  lets have a chat ]; a : [i know right]
q : [whats up with raised fist but represents racist franchise]; a : [did she even know that]
q : [yes but unfortunately in our unk and unk we killed them off]; a : [because they are the same way to me]
q : [if theres one thing i miss most about being a new member its getting a rose during unk unk ]; a : [haha i just dont know what to do that i have a lot of them]
q : [didnt you jump in halfway down the twitter thread pot meet unk]; a : [dont you know]
q : [unk of american adults dont own any guns]; a : [not the problem of the constitution]
q : [shes a cheap unk from unk cuba cheap porn star her childs father is a drug lord]; a : [what is the best thing ive ever seen it before i was in the middle years]
q : [thank you unk]; a : [youre welcome ]
q : [if we know that dying our food can cause cancer why are we still putting unk in things stop it]; a : [ dont be a bully without a liar]
q : [brad pitt unk fully with unk inves

In [24]:
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz '

def handleQueryFunc(line):
	line=line.lower()
	line=data.filter_line(line, EN_WHITELIST)

	qtokenized = [line.split(' ')]
	idx_q, idx_a = data.zero_pad(qtokenized, [[]], metadata['w2idx']) 
	return idx_q

In [ ]:
while(True):
    query = input("user :")
    idx_q = handleQueryFunc(query)
    #print (idx_q)
    output = model.predict(sess, idx_q.T)
    #print (output)
    q = data_utils.decode(sequence=idx_q[0], lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=output[0], lookup=metadata['idx2w'], separator=' ').split(' ')
    
    print('***\n query : {0}; \n reply : {1}'.format(q, ' '.join(decoded)))

user :hi
***
 query : hi; 
 reply : hows the weekend
user :how is trump
***
 query : how is trump; 
 reply : and trump is a disgrace to the debate
user :how did he win
***
 query : how did he win; 
 reply : i dont know what he was doing
user :how did trump win
***
 query : how did trump win; 
 reply : and he has a lot of his life
user :today's my birthday
***
 query : todays my birthday; 
 reply : happy birthday 
user :thank you
***
 query : thank you; 
 reply : youre welcome 
